# http://bit.ly/crawl_web_ppim

In [ ]:
# Installing Modules for Google Colab
!wget https://raw.githubusercontent.com/taufikedys/ppim/master/taudata_web.py
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/slang.dic
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/stopwords_en.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/stopwords_eng.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/all_indo_man_tag_corpus_model.crf.tagger
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/kata_dasar.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/ppim/master/data/websites.txt
!pip install unidecode
!pip install textblob
!pip install sastrawi
!pip install spacy
!pip install python-crfsuite
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm
import nltk
nltk.download('popular')

In [1]:
import warnings; warnings.simplefilter('ignore')
import taudata_web as tau
import requests
import urllib.request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
from urllib3.exceptions import InsecureRequestWarning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
fSites = 'data/websites.txt'
df = open(fSites,"r",encoding="utf-8", errors='replace')
webs = df.readlines()
webs = [w.strip() for w in webs]
df.close()
print(webs)

['http://www.eramuslim.com', 'https://www.islamramah.co']


In [3]:
headers = tau.getHeaders()
headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [4]:
data = []
for url in webs:
    req = Request(url, headers = headers) 
    doc = urlopen(req).read()
    doc = tau.text_from_html(doc)
    data.append(doc)

In [9]:
print(data[0])

             Indeks  Foto  Video                       Depan  Berita   Editorial  Dunia Islam  Palestina  Silaturrahim  Nasional  Info Umat  Historia  Catatan Redaksi    Oase Iman   Kisah  Dakwah Mancanegara    Peradaban   Ekonomi Syariah  Pemikiran Islam  Bercermin Pada Salaf  Sirah Tematik  Mukjizat Quran & Sunnah  Tafsir Fi Zhilalil Qur’an    Fokus   Analisa  Laporan Khusus  Tahukah Anda  Bincang-Bincang    Muslimah   Wanita Bicara  Inspirasi Wanita  Qur’anic Parenting  Lifestyle Muslimah  Kuliner    Hikmah   Tafakur  Bisnis itu Jihad    Pendidikan & Keluarga   Pendidikan  Keluarga  Konsultasi Pend. & Keluarga    Bisnis   Info Bisnis  Resensi Buku  Info Produk  Resensi  E-Market  E-Plaza    Ustadz   Ustadz Menjawab  Nasehat Ulama  Khutbah Jumat    Suara Langit   Penetrasi Ideologi  Ringan Berbobot  Menuju Kehidupan Sejati  Undangan ke Surga    Novel  Konsultasi   Klinik Sehat  Di Balik Konspirasi  Konsultasi Keluarga  Thibbun Nabawi  Konsultasi Arsitektur  Konsultasi Zakat  Wakaf   

# PreProcessing

In [10]:
fSlang = 'data/slang.dic'
stops, lemma = tau.LoadStopWords(lang='id')
fix = tau.loadCorpus(file = fSlang, sep=':')

In [11]:
cleanData = []
for i, web in enumerate(data):
    clean_web = tau.cleanText(web, fix=fix, lemma=lemma, stops = stops, min_charLen = 2, max_charLen = 12, fixTag= False, fixMix=True)
    cleanData.append(clean_web)

In [12]:
cleanData[0][:1000]

'indeks foto video berita editorial dunia islam palestina silaturrahim nasional informasi umat historia catatan redaksi oase iman kisah dakwah mancanegara peradaban ekonomi syariah pemikiran islam bercermin salaf sirah tematik mukjizat quran sunnah tafsir fi zhilalil fokus analisa laporan khusus tahukah bincang-bincang muslimah wanita bicara inspirasi wanita parenting lifestyle muslimah kuliner hikmah tafakur bisnis jihad pendidikan keluarga pendidikan keluarga konsultasi pend. keluarga bisnis informasi bisnis resensi buku informasi produk resensi market plaza ustadz ustadz nasehat ulama khutbah jumat suara langit penetrasi ideologi ringan berbobot kehidupan sejati undangan kah surga novel konsultasi klinik sehat konspirasi konsultasi keluarga thibbun nabawi konsultasi arsitektur konsultasi zakat wakaf senin jumadil awwal januari senin sabar balasannya kemuliaan penjelasannya senin tingkatan ikhlas beribadah senin jalan terang menguak pemilihan curang senin periode jokowi tidak senin a

# Document Summarization Technique

In [15]:
import warnings; warnings.simplefilter('ignore')
from gensim.summarization import summarize

summarize(data[0], split=True, ratio=0.05)

['Ular Masuk ke Celana Anak Tidur Terekam CCTV         Senin, 20/01/2020 11:30:06  Sains Modern Buktikan Kurma Nabi (Ajwa) Mampu Mencegah Kanker         Senin, 20/01/2020 11:00:11  Ini Daftar Aneka Subsidi Energi yang Dicabut di Era Jokowi         Senin, 20/01/2020 10:42:03  INILAH SAHABAT WANITA SEHAT MINYAK ZAITUN DAN HABBATUSSAUDA         Senin, 20/01/2020 10:31:37  Berkahi Umur Kita dengan MEMILIKI SAUNG VILLA KAMPUNG SAHABAT QUR’AN           Terbaru  Terpopuler      Sabar Balasannya Kemuliaan, Ini Penjelasannya    3 Tingkatan Ikhlas dalam Beribadah    Jalan Terang Menguak Pemilu yang Curang    Periode Kedua Jokowi Tak Sampai 2024    Anies Baswedan: Monas Akan Lebih Hijau    Merinding!',
 'Ular Masuk ke Celana Anak Tidur Terekam CCTV    Sains Modern Buktikan Kurma Nabi (Ajwa) Mampu Mencegah Kanker    Ini Daftar Aneka Subsidi Energi yang Dicabut di Era Jokowi    INILAH SAHABAT WANITA SEHAT MINYAK ZAITUN DAN HABBATUSSAUDA   Selengkapnya »       Periode Kedua Jokowi Tak Sampai 2024   

In [16]:
summarize(cleanData[0], split=True, ratio=0.05)

['ular masuk kah celana anak tidur terekam cctv senin sains modern buktikan kurma nabi ajwa mencegah kanker senin daftar aneka subsidi energi dicabut era jokowi senin sahabat wanita sehat minyak zaitun habbatussauda senin berkahi umur memiliki saung villa kampung sahabat terbaru terpopuler sabar balasannya kemuliaan penjelasannya tingkatan ikhlas beribadah jalan terang menguak pemilihan curang periode jokowi tidak anies baswedan monas hijau merinding.',
 'ular masuk kah celana anak tidur terekam cctv sains modern buktikan kurma nabi ajwa mencegah kanker daftar aneka subsidi energi dicabut era jokowi sahabat wanita sehat minyak zaitun habbatussauda selengkapnya periode jokowi tidak dinilai komentar raja sunda empire ancam ridwan kamil iwan sumule harun masiku tertangkap cctv bandara soetta berbohong.']

In [17]:
from gensim.summarization import keywords

keywords(data[0]).split('\n')[:15]

['dan',
 'pemilu yang',
 'senin',
 'untuk',
 'ini',
 'islam',
 'kita dengan',
 'anak',
 'eramuslim',
 'dalam',
 'sebagai',
 'jalan',
 'nabi',
 'quran',
 'ada']

In [18]:
keywords(cleanData[0]).split('\n')[:15]

['senin',
 'anak',
 'sahabat',
 'nabi',
 'jalan',
 'kah surga',
 'sunat',
 'quran',
 'kaum kafir',
 'sejarah',
 'muhammad',
 'konsultasi',
 'rasulullah',
 'resensi buku',
 'memiliki']